In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from datetime import datetime, timedelta

In [3]:
project_root = '/user_data/mmhender/featsynth/'

expt_name = 'pilot4'

data_folder = os.path.join(project_root, 'online_data', expt_name)
expt_design_folder = os.path.join(project_root, 'expt_design', expt_name)

preproc_folder = os.path.join(data_folder, 'preproc')
if not os.path.exists(preproc_folder):
    os.makedirs(preproc_folder)
    
folders = os.listdir(data_folder)
folders = [f for f in folders if os.path.isdir(os.path.join(data_folder,f))]
folders = [f for f in folders if 'data' in f and 'old' not in f]
# folders = [folders[0]]
folders

['data_exp_113782-v4']

In [4]:
raw_data = pd.DataFrame()
raw_worker_ids = pd.DataFrame()

for ff, folder in enumerate(folders):
    
    string1 = 'task'
    subfolder = os.path.join(data_folder, folders[ff])
    files = os.listdir(subfolder)
    files = [f for f in files if string1 in f and '.csv' in f]
    task_filename = os.path.join(subfolder,files[0])
    print(task_filename)

    r = pd.read_csv(task_filename)
    raw_data = pd.concat([raw_data, r])
    
    
    string2 = '89gk'
    subfolder = os.path.join(data_folder, folders[ff])
    files = os.listdir(subfolder)
    files = [f for f in files if string2 in f and '.csv' in f]
    if len(files)>0:
        worker_id_filename = os.path.join(subfolder,files[0])
        r = pd.read_csv(worker_id_filename)
        raw_worker_ids = pd.concat([raw_worker_ids, r])
    
raw_data.set_index(np.arange(raw_data.shape[0]));
raw_worker_ids.set_index(np.arange(raw_worker_ids.shape[0]));


/user_data/mmhender/featsynth/online_data/pilot4/data_exp_113782-v4/data_exp_113782-v4_task-sq8b.csv


/home/mmhender/imstat_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
public_ids = np.array(raw_data['Participant Public ID'])
public_ids = [p if isinstance(p,str) else '' for p in public_ids]
public_ids = np.unique(public_ids)
public_ids = np.array([p for p in public_ids if len(p)>1])

is_good = []
for pub_id in public_ids:
    inds = np.where(raw_data['Participant Public ID']==pub_id)[0]
    d = raw_data.iloc[inds]
    
    code = np.array(d['Participant Completion Code'])
    has_code = np.any([isinstance(c,str) for c in code])
    
    is_good += [has_code]

In [6]:
time_format_str = '%d/%m/%Y %H:%M:%S'

def adjust_datetime_str(date_str, time_format_str, hours):
    date_num = datetime.strptime(date_str, time_format_str)
    date_num_adj = date_num + timedelta(hours=-8)
    date_str_adj = datetime.strftime(date_num_adj,  time_format_str)
    return date_str_adj

def make_subject_df(public_ids):

    colnames = ['Gorilla public ID', 'Survey Completion Code', \
                'Worker ID mTurk',\
                'Expt Start Time (PST)', 'Expt End Time (PST)', 'Total number of trials', \
               'Experiment version num', 'Task version num']

    df = pd.DataFrame(columns=colnames)

    for ii, pub_id in enumerate(public_ids):

        inds = np.where(raw_data['Participant Public ID']==pub_id)[0]

        d = raw_data.iloc[inds]

        trial_nums = np.array(d['trial_in_run'])
        trial_nums = trial_nums[~np.isnan(trial_nums)]
        n_trials = int(len(trial_nums)/3)

        code = np.array(d['Participant Completion Code'])[0]

        # converting to PST here
        start = adjust_datetime_str(np.min(d['UTC Date']), time_format_str, -8)
        end = adjust_datetime_str(np.max(d['UTC Date']), time_format_str, -8)

        subject_inds = [pub_id==pid for pid in raw_worker_ids['Participant Public ID']]
        if sum(subject_inds)>0:
            inds = np.where(subject_inds & (raw_worker_ids['Question Key']=='worker-id'))[0][0]
            wid = raw_worker_ids['Response'].iloc[inds]
        else:
            wid = 'nan'

        task_version = np.array(d['Task Version'])[0]
        expt_version = np.array(d['Experiment Version'])[0]

        vals = np.array([[pub_id, code, wid, start, end, n_trials, expt_version, task_version]])
        df = pd.concat([df, pd.DataFrame(vals, columns=colnames, index=[ii])],\
                        axis=0)
    return df

#### Print complete subjects

In [10]:
good_ids = public_ids[np.array(is_good)]
good_df = make_subject_df(good_ids)
fn2save = os.path.join(preproc_folder,'good_sub_list.csv')
good_df.to_csv(fn2save)
# sort_order = np.argsort(np.array(good_df['Expt End Time (PST)']))
sort_order = np.argsort(np.array(good_df['Expt Start Time (PST)']))
# good_df.iloc[sort_order]
good_df

,Gorilla public ID,Survey Completion Code,Worker ID mTurk,Expt Start Time (PST),Expt End Time (PST),Total number of trials,Experiment version num,Task version num
0,0q0shv3m,ifgqoo,AZ69TBTDH7AZS,26/01/2023 18:06:24,26/01/2023 19:04:32,1000,4.0,1.0
1,15owyc9p,wpbzbv,AND64IJKXZZMG,21/01/2023 11:53:53,21/01/2023 13:24:07,1000,4.0,1.0
2,1aw5gys2,fsxsiz,A2YC6PEMIRSOAA,19/01/2023 09:57:20,19/01/2023 10:59:27,1000,4.0,1.0
3,1ql190pa,scplmj,A354LBXNXIKTKN,21/01/2023 16:01:03,21/01/2023 16:57:41,1000,4.0,1.0
4,2wny9ec4,oyvshm,A2F5SKVBQQXFX0,21/01/2023 12:35:33,21/01/2023 13:32:54,1000,4.0,1.0
5,380cpo0y,nlposs,AMCT1XHLT5I1M,26/01/2023 17:19:32,26/01/2023 18:26:30,1000,4.0,1.0
6,6g49pnz7,icvwsg,AF318MABD1U0S,27/01/2023 10:33:45,27/01/2023 11:32:01,1000,4.0,1.0
7,6swid0id,hpiimu,A256FHXGSY0E5D,19/01/2023 10:09:34,19/01/2023 11:08:23,1000,4.0,1.0
8,7vtju91h,txwlud,A2MWAXV1YRK5GH,19/01/2023 10:24:02,19/01/2023 11:36:16,1000,4.0,1.0
9,9mjm77fg,mcskel,A2V27A9GZA1NR2,27/01/2023 10:28:22,27/01/2023 12:55:19,1000,4.0,1.0


#### Print incomplete subjects

In [8]:
incomplete_ids = public_ids[~np.array(is_good)]
incomplete_df = make_subject_df(incomplete_ids)
fn2save = os.path.join(preproc_folder,'incomplete_sub_list.csv')
incomplete_df.to_csv(fn2save)
incomplete_df

,Gorilla public ID,Survey Completion Code,Worker ID mTurk,Expt Start Time (PST),Expt End Time (PST),Total number of trials,Experiment version num,Task version num
0,ucat4e0x,nan,A2NAKIXS3DVGAA,19/01/2023 09:58:25,19/01/2023 10:31:00,500,4.0,1.0


In [11]:
super_names_eachset = []
basic_names_eachset = []

for which_cb in [1,2]:
    info_filename = os.path.join(expt_design_folder, 'trial_info_counterbal%d.csv'%which_cb)
    info = pd.read_csv(info_filename)
    
    super_names = np.unique(info['super_name'])
    basic_names = []
    for sup in super_names:
        basic = np.unique(np.array(info['basic_name'][info['super_name']==sup]))
        basic_names += [basic]
        
    super_names_eachset += [super_names]    
    basic_names_eachset += [basic_names]
    
n_supcat = len(super_names_eachset[0])
n_bascat = len(basic_names_eachset[0][0])
n_supcat, n_bascat

(10, 10)

In [12]:
def preproc_data(data):
    
    start_run_inds = np.where(~np.isnan(data['subject_id_rnd']))[0][0::2]
    stop_run_inds = np.where(~np.isnan(data['subject_id_rnd']))[0][1::2]
    finish_inds = stop_run_inds+1

    # print(data.shape)
    # print(finish_inds)
    run_acc = np.array(data['total_acc'])[finish_inds]
    run_avg_rts = np.array(data['average_rt'])[finish_inds]
    print('run accuracies, avg RTs:')
    print(run_acc, run_avg_rts)
    
    run_numbers = np.array(data['run_number'])[start_run_inds].astype(int)
   
    which_cb = int(np.array(data['which_counterbal'])[start_run_inds[0]])
    # which_cb = 1
    print(which_cb)
    
    info_filename = os.path.join(expt_design_folder, 'trial_info_counterbal%d.csv'%which_cb)
    info = pd.read_csv(info_filename)

    n_trials = np.sum(data['is_stim'])
    assert(info.shape[0]==n_trials)

    trial_data = info
    rts = np.array(data['Reaction Time'])
    stim_rts = rts[data['is_stim']==True]
    iti_rts = rts[data['is_iti']==True]

    trial_data['run_number'] = np.repeat(run_numbers, int(n_trials/len(run_numbers)))
    
    stimulus_dur_ms = 300

    rts = stim_rts
    rts[np.isnan(stim_rts)] = iti_rts[np.isnan(stim_rts)] + stimulus_dur_ms

    trial_data['rt'] = rts

    resp = np.array(proc_resp_strs(data['Response']))

    stim_resp = resp[data['is_stim']==True]
    iti_resp = resp[data['is_iti']==True]
    resp = stim_resp
    resp[np.isnan(stim_resp)] = iti_resp[np.isnan(stim_resp)]

    resp[np.isnan(resp)] = -1
    trial_data['resp'] = resp.astype(int)

    correct_resp = np.array(data['correct_response'])
    correct_resp = correct_resp[data['is_stim']==True].astype(int)

    # double check that trial sequence is correct between data and trial info csv
    correct_resp_check = 2-info['target_present'].astype(int)
    # print(correct_resp_check, correct_resp)
    # print(correct_resp_check==correct_resp)
    # print(np.where(~(correct_resp_check==correct_resp)))
    assert(np.all(correct_resp_check==correct_resp))

    trial_data['correct_resp'] = correct_resp

    trial_data['correct'] = trial_data['correct_resp']==trial_data['resp']

    return trial_data, info

def proc_resp_strs(resp, poss_resp = ['1','2']):

    resp = [r if isinstance(r, str) else str(r) for r in resp]
    proc_resp = np.array([int(r) if r in poss_resp else np.nan for r in resp])

    return proc_resp

In [13]:
subject_ids = good_ids
n_subjects = len(subject_ids)

In [14]:
subject_ids.shape

(38,)

In [15]:
trial_data_all = pd.DataFrame()
good_subject_count = 0
for si, ss in enumerate(subject_ids):

    inds = np.where(raw_data['Participant Public ID']==ss)[0]
    d = raw_data.iloc[inds,:]
    
    # sort by time. sometimes they are out of order, sometimes not...
    t = d['Local Timestamp']
    
    d = d.iloc[np.argsort(t)]
    
    which_cb = np.array(d['which_counterbal'])
    which_cb = int(which_cb[~np.isnan(which_cb)][0])

    pub_id = np.array(d['Participant Public ID'])[0]
    print('\nsubject %d, id: %s'%(si, pub_id))
    print('counterbalance cond: %d'%which_cb)
    
    trial_data, info = preproc_data(d)
    
    # as quality control, check accuracy on the intact condition. subjects
    # should all be above 0.50 on this since it's easy.
    intact_data = trial_data[trial_data['image_type']=='orig']
    intact_acc = np.mean(intact_data['resp']==intact_data['correct_resp'])

    print('intact acc: %.2f'%intact_acc)
    if intact_acc<0.50:
        
        print('skipping this subject')
        continue
        
    else:
        
        good_subject_count += 1
        trial_data['subject']=np.full(fill_value=good_subject_count, shape=[trial_data.shape[0],])

        trial_data['which_cb']=np.full(fill_value=which_cb, shape=[trial_data.shape[0],])
        trial_data['gorilla_pub_id']=np.full(fill_value=ss, shape=[trial_data.shape[0],])
        trial_data['worker_id']=np.full(fill_value=good_df['Worker ID mTurk'].iloc[si], shape=[trial_data.shape[0],])

        trial_data_all = pd.concat([trial_data_all, trial_data])
    
trial_data_all.set_index(np.arange(trial_data_all.shape[0]));
fn2save = os.path.join(preproc_folder, 'preproc_data_all.csv')
trial_data_all.to_csv(fn2save)


subject 0, id: 0q0shv3m
counterbalance cond: 2
run accuracies, avg RTs:
[68. 73. 62. 69. 65. 66. 61. 71. 64. 68.] [603. 616. 575. 576. 642. 671. 657. 635. 646. 618.]
2
intact acc: 0.80

subject 1, id: 15owyc9p
counterbalance cond: 1
run accuracies, avg RTs:
[64. 60. 61. 64. 65. 63. 66. 64. 55. 55.] [1014.  920. 1023. 1013.  918.  961.  903.  912.  872.  967.]
1
intact acc: 0.73

subject 2, id: 1aw5gys2
counterbalance cond: 1
run accuracies, avg RTs:
[69. 71. 75. 73. 75. 83. 77. 71. 70. 71.] [ 836.  904. 1027.  835.  814.  882.  937.  958.  958.  900.]
1
intact acc: 0.88

subject 3, id: 1ql190pa
counterbalance cond: 1
run accuracies, avg RTs:
[52. 50. 44. 54. 40. 42. 35. 50. 52. 51.] [467. 523. 505. 555. 568. 468. 580. 787. 878. 779.]
1
intact acc: 0.43
skipping this subject

subject 4, id: 2wny9ec4
counterbalance cond: 2
run accuracies, avg RTs:
[51. 43. 34. 43. 43. 42. 43. 43. 42. 46.] [880. 710. 857. 979. 832. 824. 881. 800. 828. 844.]
2
intact acc: 0.42
skipping this subject

subje